----
## **Gri Search - SARIMAX**
----

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from itertools import product
from sklearn.metrics import mean_squared_error


In [ ]:
df = pd.read_csv('dados_processados.csv', index_col='date', parse_dates=True)

features = [
    'impressions', 'clicks', 
]

y = df['spend']
X = df[features] 

In [ ]:
p = d = q = range(0, 4)
P = D = Q = range(0, 4)
s = [7]  

pdq = list(product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], s[0]) for x in product(P, D, Q)]


In [ ]:
import warnings
warnings.filterwarnings("ignore")

best_aic = np.inf
best_order = None
best_seasonal_order = None

for param in pdq:
    for seasonal_param in seasonal_pdq:
        try:
            mod = sm.tsa.SARIMAX(
                y,
                exog=X,
                order=param,
                seasonal_order=seasonal_param,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            results = mod.fit(disp=False)
            
            if results.aic < best_aic:
                best_aic = results.aic
                best_order = param
                best_seasonal_order = seasonal_param

            print(f'ARIMA{param}x{seasonal_param}7 - AIC:{results.aic:.2f}')
        except:
            continue

print('Melhor combinação:')
print('Order:', best_order)
print('Seasonal Order:', best_seasonal_order)
print('AIC:', best_aic)


### Testes com diferentes features

```
Melhor combinação:
Order: (0, 1, 1)
Seasonal Order: (0, 0, 1, 7)
AIC: 3477.6358896896745
```

```
Melhor combinação:
Order: (1, 0, 1)
Seasonal Order: (0, 0, 1, 7)
AIC: 3537.5273274943747
```

```
🏆 Melhor combinação:
Order: (0, 1, 0)
Seasonal Order: (1, 1, 3, 7)
AIC: 3374.5589498122676
```

In [ ]:
# Modelo final com os melhores parâmetros
model = sm.tsa.SARIMAX(
    y,
    exog=X,
    order=best_order,
    seasonal_order=best_seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)

results = model.fit()
print(results.summary())


----
## **Gri Search - XGBoost**
----

In [ ]:
df = pd.read_csv('dados_processados.csv', index_col='date', parse_dates=True)

features = [
    'impressions', 'clicks', 'reach', 'cpc', 'ctr',
    'dia_semana_cos','dia_semana_sin', 'dia_do_mes', 
    'mes_cos', 'mes_sin', 'semana_do_ano',
    'spend_lag_1', 'spend_lag_7',
    'spend_media_movel_7d',
    'eh_feriado', 
]

y = df['spend']
X = df[features]  

In [ ]:
!pip install xgboost

In [ ]:

split_date = X.index[-14]

X_train = X[X.index < split_date]
y_train = y[X.index < split_date]

X_valid = X[X.index >= split_date]
y_valid = y[X.index >= split_date]


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Modelo base
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# Parâmetros para testar
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 3, 4],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.7, 1],
}

from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=3)

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=tscv,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Melhores hiperparâmetros:", grid_search.best_params_)
print("Melhor RMSE (negativo):", grid_search.best_score_)


### Testes com diferentes parâmetros

```
Melhores hiperparâmetros: {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 1000, 'subsample': 0.7}
Melhor RMSE (negativo): -60.31514982527961
```

```
Melhores hiperparâmetros: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 200, 'subsample': 0.7}
Melhor RMSE (negativo): -63.07732239013345
```

In [ ]:
# Melhor modelo treinado
best_xgb = grid_search.best_estimator_

o
y_pred = best_xgb.predict(X_valid)

rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("Validação RMSE:", rmse)

# Visualizar
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(y_valid.index, y_valid, label='Real')
plt.plot(y_valid.index, y_pred, label='Previsto')
plt.legend()
plt.title('XGBoost - Previsão vs Real')
plt.show()
